In [2]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

In [3]:
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [20]:
max_length = 16
num_beams = 4
num_return_sequences = 3

gen_kwargs = {"max_length": max_length, "num_beams": num_beams, "num_return_sequences": num_return_sequences}
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds

In [16]:
predict_step(['Image1.png'])

['a man kicking a soccer ball on a field',
 'a soccer player in action on the field',
 'a man kicking a soccer ball in the air']

In [21]:
predict_step(['Image2.png'])

['a woman is standing in a field with a horse',
 'a woman standing in a field with a horse',
 'a woman is standing in the middle of a field']

In [22]:
predict_step(['Image3.png'])

['a collage of photos showing a woman holding a sign',
 'a collage of photos showing a woman and a man',
 'a collage of photos showing a woman in a pink dress and a man']